In [203]:
!


'pip.exe' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:


_PCVERSION_=True


if _PCVERSION_:
    basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub/_EarthQuake/gpos_lin"
else:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    basedir="/content/drive/My Drive/001_AI/_EarthQuake/gpos_lin"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------------scikit import 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#--------------

def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")


def checkvalues(df,columnname,key):
    print(f"{columnname} ellenőrzése !")
    numok=0
    numerr=0
    for i in df.index:
        if df.at[i,columnname] in key:
            #print(df.at[i,columnname])
            numok+=1
            pass
        else:
            numerr+=1
            print(df.at[i,columnname],end=", ")
    sumall=numok+numerr
    print(f"\n{sumall} mintából {numerr} db nem volt megfelelő")


def createcolumn(df,columnname,keys):
    print(f"{columnname} cseréje megy")
    for key in keys:
        df[keys[key]]=0.0
    for key in keys:
        for i in df.index:
            if df.at[i,columnname]==key:
                df.at[i,keys[key]]=1.0


def create_dict(idx,list):
    o={}
    for i in list:
        o[i]=idx+"_"+i
    return o

def create_base_data(df):
    t=['n', 't', 'o']
    columnname="land_surface_condition"
    key=create_dict(columnname,t)


    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t= ['h', 'w', 'i', 'r', 'u']
    columnname="foundation_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['q', 'n', 'x']
    columnname="roof_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['z', 'v', 'f', 'm', 'x']
    columnname="ground_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['q', 's', 'j', 'x']
    columnname="other_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['j', 's', 't', 'o']
    columnname="position"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['c', 's', 'f', 'd', 'm', 'a', 'q', 'u', 'n', 'o']
    columnname="plan_configuration"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=['a', 'w', 'r', 'v']
    columnname="legal_ownership_status"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    # level 1: 0-30, level 2: 0-1427, level 3: 0-12567.
    # level1:0--30
    # level2: 0.0000-----------0.9999
    # level3: 0.000000000------0.000099999
    l1=df.geo_level_1_id
    l2=(df.geo_level_2_id/1427*9999)/10000
    l3=(df.geo_level_2_id/12567*99999)/1000000000
    df["geopos"]=l1+l2+l3
    return(df)



def kill_columns(df):
    notkey=["Unnamed: 0","building_id","legal_ownership_status","geo_level_1_id",	"geo_level_2_id",	"geo_level_3_id", "land_surface_condition",	"foundation_type",	"roof_type",	"ground_floor_type",	"other_floor_type",	"position",	"plan_configuration"]
    for i in df.columns:
        #print(i)
        if i in notkey:
            df.drop(columns=[i], inplace=True)
    return df







In [ ]:
def check_similarity_np(np1,np2):
    db=0
    maxi=0
    for ind, i1 in enumerate(np1):
        maxi+=1
        i2=np2[ind]
        if i1!=i2:
            db+=1
    #print(f"{ind}. eset:  {i:3},{i2:3}")
    print(f"hiba:{db} max:{maxi} -- error:{db/maxi*100.0 : 2.6} good %:{100-db/maxi*100.0 : 2.6} %")

In [ ]:
'''
features_train=basedir+"/orig/train_values.csv"
labels_train=basedir+"/orig/train_labels.csv"
features_predict=basedir+"/orig/test_values.csv"

X_train=pd.read_csv(features_train)
y_train=pd.read_csv(labels_train)
X_pred=pd.read_csv(features_predict)
#
X_pred_conv=create_base_data(X_pred)
X_train_conv=create_base_data(X_train)
#
X_train_ok=kill_columns(X_train_conv)
X_pred_ok=kill_columns(X_pred_conv)
y_train_ok=kill_columns(y_train)

X_train_ok.to_csv(basedir+"/tmp/X_tran_ok.csv",index=False)
X_pred_ok.to_csv(basedir+"/tmp/X_pred_ok.csv",index=False)
y_train_ok.to_csv(basedir+"/tmp/y_train_ok.csv",index=False)




#f=open(basedir+"/tmp/similarity.csv","a")
X_train_ok=pd.read_csv(basedir+"/tmp/X_tran_ok.csv",)
X_pred_ok=pd.read_csv(basedir+"/tmp/X_pred_ok.csv")
y_train_ok=pd.read_csv(basedir+"/tmp/y_train_ok.csv")

X_train_ok.head()
X_pred_ok.head()
#print("Ready")
'''

In [200]:
X_train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,plan_configuration_a,plan_configuration_q,plan_configuration_u,plan_configuration_n,plan_configuration_o,legal_ownership_status_a,legal_ownership_status_w,legal_ownership_status_r,legal_ownership_status_v,geopos
0,2,30,6,5,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.341245
1,2,10,8,7,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.630638
2,2,10,5,5,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.254357
3,2,10,6,5,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.292896
4,3,30,8,9,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.091793


In [ ]:
X_pred_ok=kill_columns(X_pred_ok)
X_train_ok=kill_columns(X_train_ok)
y_train_ok=kill_columns(y_train_ok)

In [ ]:
print(type(X_pred_ok))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler1=StandardScaler()
X_pred_scale=scaler1.fit_transform(X_pred_ok)

scaler2=StandardScaler()
X_train_scale=scaler2.fit_transform(X_train_ok)
"""
scaler3=StandardScaler()
y_train_scale=scaler3.fit_transform(y_train_ok)
"""
y_train_np=y_train_ok.to_numpy()

# szétszedjük a train és test részekre
from sklearn.model_selection import train_test_split
#X_train_train, X_train_test,y_train_train, y_train_test  = train_test_split( X_train_scale, y_train_scale, test_size=0.10, random_state=0)
X_train_train, X_train_test,y_train_train, y_train_test  = train_test_split( X_train_scale, y_train_np, test_size=0.10, random_state=0)




In [ ]:
print(type(y_train_train))

In [ ]:
# *************
# * LDA       *
# *************

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2, solver="svd", tol=0.000000001)
lda.fit(X_train_train, y_train_train)

X_train_lda = lda.transform(X_train_train)
#y_train = lda.transform(y_train_train)
X_test_lda=lda.transform(X_train_test)






In [ ]:
# OTLET:
# LDA térben keressünk outliereket... 
# ha nincs túl sok tanítsuk a modellt az inlierekre
# 
#  
from sklearn.ensemble import IsolationForest

rng=1
#------------------------------
outliers_fraction=0.01
#------------------------------
clf=IsolationForest(contamination=outliers_fraction, random_state=rng)
clf.fit(X_train_lda)
outlier_predict=clf.predict(X_train_lda)


print("Outlier Detection ENDED")



In [ ]:
print(outlier_predict)

In [ ]:
outlier_index=[]
for inx,i in enumerate(X_train_lda):
    if outlier_predict[inx]==-1:
        outlier_index.append(inx)

In [ ]:
print(type(y_train_train))

In [ ]:
def outlierDropfrom_df(df,inxlist):
    a=df
    out=a.drop(inxlist,axis=0)
    return(out)

In [ ]:
def outlierDropfrom_numpyarray(na,inxlist):
    
    out=np.delete (na ,inxlist, axis=0)
    return(out)

In [ ]:
X_train_train_out=X_train_train
y_train_train_out=y_train_train
#print(X_train_train_out)
#print(outlier_predict)

print("Drop2")

y_train_train_out=outlierDropfrom_numpyarray(y_train_train,outlier_predict)
print(len(y_train_train_out)," .. ",len(outlier_predict))
#print("Drop1",len(X_train_lda))
X_train_train_out=outlierDropfrom_numpyarray(X_train_lda,outlier_predict)
print(len(X_train_train_out)," .. ",len(outlier_predict))






In [ ]:
print(y_train_train)

In [ ]:
# XGBClassifier
'''
from xgboost import XGBClassifier  # 72.09
# max_depth=10 : 72.79857561664441

knn = XGBClassifier(verbosity=3,max_depth = 13,n_estimators=100,loss="deviance", criterion="mae")

print("Fit: Inlier betanitás Start--")

knn.fit(X_train_train_out, y_train_train_out)

print("Fit End - Prediction Start ")
'''

In [176]:
print(y_train_train_out)

[[2]
 [2]
 [2]
 ...
 [3]
 [3]
 [3]]


In [196]:
from sklearn.neural_network import MLPClassifier
knn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,), random_state=1, verbose=True, max_iter=1000, tol=0.0000001)
#knn.fit(X_train_train_out, y_train_train_out)
knn.fit(X_train_train, y_train_train)

print("Fit End - Prediction Start ")


Fit End - Prediction Start 


In [197]:
print(X_train_train_out)

[[-1.07318179 -0.22064278]
 [ 0.36871787 -0.07171722]
 [-1.24793817 -1.35487209]
 ...
 [ 0.4365659  -0.38185691]
 [ 0.25811703  1.4176803 ]
 [ 0.99633896  1.92926339]]


In [198]:
# test the traing
#------------------------------------

y_pred_train=knn.predict(X_train_train)

#------------------------------------

In [199]:
check_similarity_np(y_pred_train,y_train_train)

hiba:74285 max:234540 -- error: 31.6726 good %: 68.3274 %


In [174]:
# the prediction
#------------------------------------

y_pred_test=knn.predict(X_test_lda)

#------------------------------------

In [179]:
check_similarity_np(y_pred_test,y_train_test)

hiba:10381 max:26061 -- error: 39.8335 good %: 60.1665 %


In [ ]:
# futtass ez felett !!

In [ ]:
#outfile generation
y_pred_ok=knn.predict(X_pred_ok)
print("Prediction End")

X_pred_bd=pd.read_csv(basedir+"/orig/test_values.csv")

y_pred_ok_int=[]
for i in range(len(y_pred_ok)):
    y_pred_ok_int.append(int(y_pred_ok[i]))

In [ ]:
buildingid=X_pred_bd["building_id"]
head2=y_pred_ok_int


outdf=pd.DataFrame(data={"damage_grade":y_pred_ok_int} ,index=buildingid)
outdf.index.name="building_id"


outdf.head()
st=24
sts=str(st)
outdf.to_csv(basedir+"/out/submission_"+sts+"_xgboost.csv")
print()
print(basedir+"/out/submission_"+sts+"_xgboost.csv")

In [ ]:
if not( _PCVERSION_):
    !head "/content/drive/My Drive/001_AI/_EarthQuake/gpos_lin/out/submission_24_xgboost.csv"